# Item-based Models for Movielens

In this file we are going to proceed with the building in the models. Machine learning models to build recommender systems. The models that are going to be built are Collaborative filtering using Item-based rating prediction (ItemKNN) and Item-based classification (ItemKNN).

Collaborative filtering is a technique used in recommendation systems to predict or classify items based on the preferences or behavior of similar users or items. Item-based Collaborative Filtering (CF) focuses on the similarity between items rather than users. There are two main approaches within item-based CF: rating prediction and classification.

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to make jupyter print all outputs, not just the last one
from IPython.core.display import HTML # to pretty print pandas df and be able to copy them over (e.g. to ppt slides)

We import the necessary libraries for the modelling:

- Pandas: Data manipulation and analysis library in Python, primarily for structured data using Series and DataFrame.

- NumPy: Fundamental library for numerical computing with support for large arrays and mathematical functions.

- os: Python module for interacting with the operating system, used for file and directory operations.

In [1]:
# We import the necessary libraries
import pandas as pd
import numpy as np
import os

In [3]:
# We print the directory where the file is located
print(os.getcwd())

c:\Users\Jaume\Documents\MDDB\SDM\SDfM---Jaume-and-Stijn 2\SDfM---Jaume-and-Stijn


In [4]:
# We set the directory to the cleaned folder
os.listdir(os.path.join('.', 'cleaned'))

['final_sample30_parquet',
 'final_sample50_parquet',
 'final_sample5_parquet',
 'movielens_parquet',
 'netflix_parquet',
 'sample_tenth_netflix',
 'unpacked_reviews_df_100k.parquet']

In [5]:
# We read the final_sample file and store it in a dataframe
df = pd.read_parquet('cleaned/unpacked_reviews_df_100k.parquet')

In [6]:
# We print shape of the dataframe
df.shape

(102657, 27)

In [7]:
# We print the first 5 rows of the dataframe
df.head()

,movieId,title,year,userId,rating,date,month,Adventure,Animation,Children,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,1995,1,4.0,2000-07-30,7,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,Toy Story,1995,5,4.0,1996-11-08,11,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,Toy Story,1995,7,4.5,2005-01-25,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,Toy Story,1995,15,2.5,2017-11-13,11,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,Toy Story,1995,17,4.5,2011-05-18,5,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# We print the columns of the dataframe
df.columns

Index(['movieId', 'title', 'year', 'userId', 'rating', 'date', 'month',
       'Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy', 'Romance',
       'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'Mystery', 'Sci-Fi',
       'War', 'Musical', 'Documentary', 'IMAX', 'Western', 'Film-Noir',
       '(no genres listed)'],
      dtype='object')

In [9]:
# We print the number of unique users and movies
print(df['userId'].nunique())
print(df['movieId'].nunique())


610
9711


In [10]:
# Check if there are userIDs that haven't rated movies at all
print(df['userId'].value_counts().sort_values(ascending=False))

# Check if there are movieIDs that haven't been rated at all
print(df['movieId'].value_counts().sort_values(ascending=False))

userId
599    2795
414    2697
474    2373
448    1863
274    1346
       ... 
257      20
406      20
278      20
595      20
147      20
Name: count, Length: 610, dtype: int64
movieId
296       484
356       335
318       319
593       283
2571      280
         ... 
101360      1
128592      1
128542      1
128991      1
193609      1
Name: count, Length: 9711, dtype: int64


In [12]:
# Check if there are NaN values in the rating column
df['rating'].isna().sum()

0

We check the number of different movies that we have in the dataset and the number of unique users we have.

In [13]:
# Print the number of unique users
print("Number of unique users:", df['userId'].nunique())

# Print the number of unique movies
print("Number of unique movies:", df['movieId'].nunique())


Number of unique users: 610
Number of unique movies: 9711


In [14]:
# print the data types of the DF
df.dtypes

movieId                        int64
title                         object
year                          object
userId                         int64
rating                       float64
date                  datetime64[ns]
month                          int32
Adventure                      int64
Animation                      int64
Children                       int64
Comedy                         int64
Fantasy                        int64
Romance                        int64
Drama                          int64
Action                         int64
Crime                          int64
Thriller                       int64
Horror                         int64
Mystery                        int64
Sci-Fi                         int64
War                            int64
Musical                        int64
Documentary                    int64
IMAX                           int64
Western                        int64
Film-Noir                      int64
(no genres listed)             int64
d

In [15]:
# We select the columns that are not genres
columns_not_genres = ['movieId', 'title', 'year', 'userId', 'rating', 'date', '(no genres listed)']
for column in df.columns:
    if column not in columns_not_genres :
        df.rename(columns={column: 'genre_' + column}, inplace=True)


In [16]:
# We print the columns of the dataframe
df.columns

Index(['movieId', 'title', 'year', 'userId', 'rating', 'date', 'genre_month',
       'genre_Adventure', 'genre_Animation', 'genre_Children', 'genre_Comedy',
       'genre_Fantasy', 'genre_Romance', 'genre_Drama', 'genre_Action',
       'genre_Crime', 'genre_Thriller', 'genre_Horror', 'genre_Mystery',
       'genre_Sci-Fi', 'genre_War', 'genre_Musical', 'genre_Documentary',
       'genre_IMAX', 'genre_Western', 'genre_Film-Noir', '(no genres listed)'],
      dtype='object')

# Item-based Rating Prediction (ItemKNN)

### Step 1: User-Item matrix construction
The first thing we have to do is build the user-item matrix:

- Choose a similarity metric to calculate the similarity between items. Common metrics include cosine similarity, Pearson correlation coefficient, and Jaccard similarity.
- Calculate the similarity between each pair of items based on the ratings provided by users. This will result in an item-item similarity matrix.

##### Libraries Used
- **pandas**: A powerful data manipulation library in Python.
- **numpy**: A library for numerical computing in Python.
- **sklearn.metrics.pairwise.cosine_similarity**: A function from scikit-learn used to compute the cosine similarity between vectors.

In [17]:
import pandas as pd
import numpy as np

# Convert userId column to integer type if needed
if df['userId'].dtype != int:
    df['userId'] = df['userId'].astype(int)

# Get all unique user IDs and movie IDs
all_user_ids = np.unique(df['userId'])
all_movie_ids = np.unique(df['movieId'])

# Determine the number of unique users and movies
num_users = len(all_user_ids)
num_movies = len(all_movie_ids)

# Create a dictionary to store ratings for each user
user_ratings = {}

# Iterate through the DataFrame and populate the user_ratings dictionary
for _, row in df.iterrows():
    user_id = row['userId']
    movie_id = row['movieId']
    rating = row['rating']
    
    # Initialize the user's ratings list if not already present
    if user_id not in user_ratings:
        user_ratings[user_id] = np.zeros(num_movies)
    
    # Assign the rating to the corresponding movie index
    user_ratings[user_id][np.where(all_movie_ids == movie_id)[0][0]] = rating

# Create user-item matrix from the dictionary
item_user_matrix = pd.DataFrame(user_ratings.values(), index=user_ratings.keys(), columns=all_movie_ids)

# Convert user-item matrix to NumPy array for faster computation
item_user_array = item_user_matrix.to_numpy()


In [18]:
item_user_matrix.head()

,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
item_user_array

array([[4. , 0. , 4. , ..., 0. , 0. , 0. ],
       [4. , 0. , 0. , ..., 0. , 0. , 0. ],
       [4.5, 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 3.5, 3.5, 0. ]])

### Train-test split

The train-val-test split is a technique used in machine learning to evaluate the performance of a model. It involves dividing the dataset into three subsets: the training set, the validation set, and the test set.

The training set is used to train the model and optimize its parameters.
The validation set is used to fine-tune the model and select the best hyperparameters.
The test set is used to evaluate the final performance of the model on unseen data.
By using a train-val-test split, we can assess the model's performance on unseen data and ensure that it generalizes well to new examples. It helps prevent overfitting and provides a more reliable estimate of the model's performance in real-world scenarios.

In [20]:
from sklearn.model_selection import train_test_split
# Split the data into training and test sets
train_val, test = train_test_split(item_user_array, test_size=0.2, random_state=42)

# Split the training set into training and validation sets
train, val = train_test_split(train_val, test_size=0.2, random_state=42)

# Print the shapes of the datasets
print("Training set shape:", train.shape)
print("Validation set shape:", val.shape)
print("Test set shape:", test.shape)

# We are also going to do the split for the matrix df
# Split the user-item matrix into training and test sets
train_val_matrix, test_matrix = train_test_split(item_user_matrix, test_size=0.2, random_state=42)

# Split the training set matrix into training and validation sets
train_matrix, val_matrix = train_test_split(train_val_matrix, test_size=0.2, random_state=42)

# We print a ' ' to give some space inbetween lines
print(' ')

# Print the shapes of the matrix datasets
print("Training set matrix shape:", train_matrix.shape)
print("Validation set matrix shape:", val_matrix.shape)
print("Test set matrix shape:", test_matrix.shape)


Training set shape: (390, 9711)
Validation set shape: (98, 9711)
Test set shape: (122, 9711)
 
Training set matrix shape: (390, 9711)
Validation set matrix shape: (98, 9711)
Test set matrix shape: (122, 9711)


In [21]:
train_matrix

,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
8,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
263,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
167,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
train

array([[0., 4., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [23]:
# Calculate cosine similarity between items using NumPy functions
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

# Calculate item-item similarity matrix for train data
#item_similarity_matrix_train = np.zeros((train.shape[1], train.shape[1]))

#for i in range(train.shape[1]):
 #   for j in range(i, train.shape[1]):
  #      item_similarity_matrix_train[i, j] = cosine_similarity(train[:, i], train[:, j])
   #     item_similarity_matrix_train[j, i] = item_similarity_matrix_train[i, j]

# Adjusting the mapping to start indexing from 0
user_id_to_index = {user_id: i for i, user_id in enumerate(item_user_matrix.index)}
index_to_user_id = {i: user_id for i, user_id in enumerate(item_user_matrix.index)}

# Create a mapping from movie IDs to indices
movie_id_to_index = {movie_id: i for i, movie_id in enumerate(item_user_matrix.columns)}
index_to_movie_id = {i: movie_id for i, movie_id in enumerate(item_user_matrix.columns)}




### Similarity Calculation utilizing Genres:



In [24]:
import numpy as np

# We create item-genre matrix
def create_item_genre_matrix(df, train):
    # Extract genre columns from the dataframe
    genre_columns = [col for col in df.columns if col.startswith('genre_')]
    
    # Extract item IDs
    item_ids = train.columns
    
    # Initialize item-genre matrix with zeros
    item_genre_matrix = np.zeros((len(genre_columns), len(item_ids)))
    
    # Fill the matrix with genre information
    for i, row in df.iterrows():
        item_id = row['movieId']  # Assuming 'movieId' is the column containing item IDs
        if item_id in item_ids:
            item_index = np.where(item_ids == item_id)[0][0]
            genres = row[genre_columns].values
            item_genre_matrix[:, item_index] = genres
    
    return item_genre_matrix

# We create the item_genre matrix with the train set and the original df
item_genre_matrix = create_item_genre_matrix(df, train_matrix)


In [25]:
item_genre_matrix

item_genre_matrix.shape

array([[11.,  5.,  5., ...,  9.,  9.,  9.],
       [ 1.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  1.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

(20, 9711)

#### Collaborative Filtering with Cosine Similarity

This code snippet demonstrates how to perform item-based collaborative filtering using cosine similarity. Collaborative filtering is a technique commonly used in recommendation systems to predict a user's preferences for items based on the preferences of similar users/items.


In [26]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def calculate_item_similarity_matrix(data):
    # Calculate cosine similarity between items
    item_similarity_matrix = cosine_similarity(data.T)
    np.fill_diagonal(item_similarity_matrix, 1)  # Set diagonal values to 1
    return item_similarity_matrix

# We calculate the item similarity using the train user-item matrix data
item_similarity_matrix_train = calculate_item_similarity_matrix(train)
print(item_similarity_matrix_train)


[[1.         0.44712915 0.32065335 ... 0.         0.         0.        ]
 [0.44712915 1.         0.28478259 ... 0.         0.         0.        ]
 [0.32065335 0.28478259 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [27]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

def calculate_item_similarity_genres(item_genre_matrix):
    # Convert the item-genre matrix to a sparse representation
    sparse_item_genre_matrix = sparse.csr_matrix(item_genre_matrix)
    
    # Calculate cosine similarity between items
    item_similarity_matrix = cosine_similarity(sparse_item_genre_matrix.T)
    
    # Set diagonal values to 1
    np.fill_diagonal(item_similarity_matrix, 1)
    
    return item_similarity_matrix

# Example usage:
# Assuming item_genre_matrix is the item-genre matrix
item_similarity_genres_matrix = calculate_item_similarity_genres(item_genre_matrix)
print(item_similarity_genres_matrix)


[[1.         0.97648079 0.96010973 ... 0.9739642  0.97785775 0.98380223]
 [0.97648079 1.         0.90924121 ... 0.93913186 0.93345729 0.93913186]
 [0.96010973 0.90924121 1.         ... 0.95636507 0.95058638 0.97761763]
 ...
 [0.9739642  0.93913186 0.95636507 ... 1.         0.98183621 0.98780488]
 [0.97785775 0.93345729 0.95058638 ... 0.98183621 1.         0.98183621]
 [0.98380223 0.93913186 0.97761763 ... 0.98780488 0.98183621 1.        ]]


In [28]:
item_similarity_genres_matrix.shape

(9711, 9711)

In [29]:
movie_id_to_index

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 12: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 18: 17,
 19: 18,
 20: 19,
 21: 20,
 22: 21,
 23: 22,
 24: 23,
 25: 24,
 26: 25,
 27: 26,
 28: 27,
 29: 28,
 30: 29,
 31: 30,
 32: 31,
 34: 32,
 36: 33,
 38: 34,
 39: 35,
 40: 36,
 41: 37,
 42: 38,
 43: 39,
 44: 40,
 45: 41,
 46: 42,
 47: 43,
 48: 44,
 49: 45,
 50: 46,
 52: 47,
 53: 48,
 54: 49,
 55: 50,
 57: 51,
 58: 52,
 60: 53,
 61: 54,
 62: 55,
 63: 56,
 64: 57,
 65: 58,
 66: 59,
 68: 60,
 69: 61,
 70: 62,
 71: 63,
 72: 64,
 73: 65,
 74: 66,
 75: 67,
 76: 68,
 77: 69,
 78: 70,
 79: 71,
 80: 72,
 81: 73,
 82: 74,
 83: 75,
 85: 76,
 86: 77,
 87: 78,
 88: 79,
 89: 80,
 92: 81,
 93: 82,
 94: 83,
 95: 84,
 96: 85,
 97: 86,
 99: 87,
 100: 88,
 101: 89,
 102: 90,
 103: 91,
 104: 92,
 105: 93,
 106: 94,
 107: 95,
 108: 96,
 110: 97,
 111: 98,
 112: 99,
 113: 100,
 116: 101,
 117: 102,
 118: 103,
 119: 104,
 121: 105,
 122: 106,
 123: 107,
 125: 108,
 126: 10

In [30]:
print(index_to_user_id)

{0: 1, 1: 5, 2: 7, 3: 15, 4: 17, 5: 18, 6: 19, 7: 21, 8: 27, 9: 31, 10: 32, 11: 33, 12: 40, 13: 43, 14: 44, 15: 45, 16: 46, 17: 50, 18: 54, 19: 57, 20: 63, 21: 64, 22: 66, 23: 68, 24: 71, 25: 73, 26: 76, 27: 78, 28: 82, 29: 86, 30: 89, 31: 90, 32: 91, 33: 93, 34: 96, 35: 98, 36: 103, 37: 107, 38: 112, 39: 119, 40: 121, 41: 124, 42: 130, 43: 132, 44: 134, 45: 135, 46: 137, 47: 140, 48: 141, 49: 144, 50: 145, 51: 151, 52: 153, 53: 155, 54: 156, 55: 159, 56: 160, 57: 161, 58: 166, 59: 167, 60: 169, 61: 171, 62: 177, 63: 178, 64: 179, 65: 182, 66: 185, 67: 186, 68: 191, 69: 193, 70: 200, 71: 201, 72: 202, 73: 206, 74: 213, 75: 214, 76: 216, 77: 217, 78: 219, 79: 220, 80: 223, 81: 226, 82: 229, 83: 232, 84: 233, 85: 234, 86: 239, 87: 240, 88: 247, 89: 249, 90: 252, 91: 254, 92: 263, 93: 264, 94: 266, 95: 269, 96: 270, 97: 273, 98: 274, 99: 275, 100: 276, 101: 277, 102: 279, 103: 280, 104: 282, 105: 283, 106: 288, 107: 290, 108: 291, 109: 292, 110: 293, 111: 298, 112: 304, 113: 307, 114: 314

In [31]:
print(user_id_to_index)

{1: 0, 5: 1, 7: 2, 15: 3, 17: 4, 18: 5, 19: 6, 21: 7, 27: 8, 31: 9, 32: 10, 33: 11, 40: 12, 43: 13, 44: 14, 45: 15, 46: 16, 50: 17, 54: 18, 57: 19, 63: 20, 64: 21, 66: 22, 68: 23, 71: 24, 73: 25, 76: 26, 78: 27, 82: 28, 86: 29, 89: 30, 90: 31, 91: 32, 93: 33, 96: 34, 98: 35, 103: 36, 107: 37, 112: 38, 119: 39, 121: 40, 124: 41, 130: 42, 132: 43, 134: 44, 135: 45, 137: 46, 140: 47, 141: 48, 144: 49, 145: 50, 151: 51, 153: 52, 155: 53, 156: 54, 159: 55, 160: 56, 161: 57, 166: 58, 167: 59, 169: 60, 171: 61, 177: 62, 178: 63, 179: 64, 182: 65, 185: 66, 186: 67, 191: 68, 193: 69, 200: 70, 201: 71, 202: 72, 206: 73, 213: 74, 214: 75, 216: 76, 217: 77, 219: 78, 220: 79, 223: 80, 226: 81, 229: 82, 232: 83, 233: 84, 234: 85, 239: 86, 240: 87, 247: 88, 249: 89, 252: 90, 254: 91, 263: 92, 264: 93, 266: 94, 269: 95, 270: 96, 273: 97, 274: 98, 275: 99, 276: 100, 277: 101, 279: 102, 280: 103, 282: 104, 283: 105, 288: 106, 290: 107, 291: 108, 292: 109, 293: 110, 298: 111, 304: 112, 307: 113, 314: 114

In [32]:
item_user_array.shape

(610, 9711)

In [33]:
item_user_matrix.shape

(610, 9711)

In [34]:
item_similarity_matrix_train.shape

(9711, 9711)

### Matrices Summation:

Once we have both similarity matrices (the one from the ratings and the one from the genres) we can add them with a weighted sumation.

In [35]:
import numpy as np

def combine_similarity_matrices(similarity_matrix1, similarity_matrix2, weight1=0.5, weight2=0.5):
    # Check if the matrices have the same shape
    if similarity_matrix1.shape != similarity_matrix2.shape:
        raise ValueError("Matrices must have the same shape.")
    
    # Combine matrices using weighted summation directly without creating an intermediate array
    np.multiply(similarity_matrix1, weight1, out=similarity_matrix1)
    np.multiply(similarity_matrix2, weight2, out=similarity_matrix2)
    np.add(similarity_matrix1, similarity_matrix2, out=similarity_matrix1)
    return similarity_matrix1

# Define weights for each matrix
weight_ratings = 0.5
weight_genres = 0.5

# Combine the similarity matrices
item_similarity_matrix_train = combine_similarity_matrices(item_similarity_matrix_train, item_similarity_genres_matrix, weight_ratings, weight_genres)

# Print or use the combined similarity matrix as needed
print("Combined Similarity Matrix:\n", item_similarity_matrix_train)


Combined Similarity Matrix:
 [[1.         0.71180497 0.64038154 ... 0.4869821  0.48892887 0.49190111]
 [0.71180497 1.         0.5970119  ... 0.46956593 0.46672865 0.46956593]
 [0.64038154 0.5970119  1.         ... 0.47818253 0.47529319 0.48880881]
 ...
 [0.4869821  0.46956593 0.47818253 ... 1.         0.49091811 0.49390244]
 [0.48892887 0.46672865 0.47529319 ... 0.49091811 1.         0.49091811]
 [0.49190111 0.46956593 0.48880881 ... 0.49390244 0.49091811 1.        ]]


### Step 2: Neighborhood Selection
- Determine the neighborhood size, i.e., the number of most similar items to consider when predicting ratings for a target item.
- Select the most similar items for each item in the dataset based on their calculated similarities. This forms the neighborhood for each item.

#### Item-Based Neighborhoods and Ratings Aggregation

This code following snippet enhances the previous item-based collaborative filtering approach by considering ratings aggregation within the item neighborhoods.

##### Steps:

1. **Defining Neighborhood Size**:
   - The variable `neighborhood_size` determines the number of most similar items to consider in the neighborhood.

2. **Initializing Data Structure**:
   - An empty dictionary `item_neighborhoods` is initialized to store the neighborhoods for each item.

3. **Iterating Over Items**:
   - For each movie in the dataset:
     - All ratings for the current movie are extracted from the DataFrame (`df`).
     - Ratings aggregation is performed. In this example, the average rating for the movie is computed, but other aggregation methods can be used.
     - The similarity scores for the current movie are retrieved from the precomputed `item_similarity_matrix`.
     - Similarity scores are sorted in descending order, and the indices of the most similar items (excluding itself) are obtained.
     - These indices are converted back to movie IDs, forming the neighborhood for the current item.
     - The neighborhood for the current item is stored in the `item_neighborhoods` dictionary.

4. **Output**:
   - `item_neighborhoods`: A dictionary where keys are movie IDs, and values are lists of movie IDs representing the neighborhood of each item. Each movie's neighborhood includes movies with similar ratings and content.

##### Note:
- This approach considers both similarity in ratings and content (as captured by cosine similarity) when building item neighborhoods.
- Aggregating ratings within item neighborhoods helps in providing more personalized recommendations.
- The choice of rating aggregation method (e.g., mean, median) can impact the quality of recommendations and may need to be adjusted based on the characteristics of the dataset and user preferences.


In [36]:
import numpy as np

def neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix, neighborhood_size=5):
    # Initialize an empty dictionary to store item neighborhoods
    item_neighborhoods = {}

    # Iterate over each item (movie) index in the dataset
    for movie_index in range(item_user_array.shape[1]):
        # Convert the item index to movie ID
        movie_id = index_to_movie_id[movie_index]

        # Extract all ratings for the current movie
        movie_ratings = item_user_array[:, movie_index]

        # Aggregate ratings (e.g., compute the mean rating)
        movie_avg_rating = np.mean(movie_ratings)

        # Retrieve similarity scores for the current movie
        similarity_scores = item_similarity_matrix[movie_index]

        # Sort similarity scores in descending order and get indices of most similar items
        most_similar_indices = np.argsort(similarity_scores)[::-1][1:neighborhood_size+1]

        # Convert indices back to movie IDs to form the neighborhood
        neighborhood = [index_to_movie_id[idx] for idx in most_similar_indices]

        # Store the neighborhood for the current item in the item_neighborhoods dictionary
        item_neighborhoods[movie_id] = neighborhood

    return item_neighborhoods

# Example usage:
item_neighborhoods_train = neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix_train)
print(item_neighborhoods_train)


{1: [3114, 260, 480, 32, 296], 2: [367, 158, 364, 480, 500], 3: [3450, 276, 762, 5, 788], 4: [359, 304, 795, 219, 310], 5: [65, 3, 762, 277, 637], 6: [733, 32, 293, 494, 112], 7: [708, 11, 552, 497, 4], 8: [271, 174, 217, 259, 484], 9: [74, 786, 634, 667, 743], 10: [380, 165, 592, 349, 316], 11: [597, 539, 440, 357, 590], 12: [879, 840, 75, 842, 656], 13: [1686, 6889, 95193, 258, 888], 14: [52, 1366, 25, 892, 896], 15: [704, 1606, 93, 303, 240], 16: [1213, 555, 1466, 4262, 1089], 17: [151, 509, 46, 224, 1680], 18: [5298, 6881, 3862, 94, 4149], 19: [344, 367, 231, 104, 333], 20: [7007, 50798, 95583, 97860, 92008], 21: [235, 440, 357, 457, 474], 22: [257, 350, 540, 230, 218], 23: [5628, 484, 325, 5423, 1785], 24: [802, 490, 416, 981, 191], 25: [36, 32, 14, 235, 16], 26: [273, 5628, 536, 41, 314], 27: [1006, 191, 33672, 91079, 71619], 28: [1497, 1733, 605, 2620, 2929], 29: [1274, 1199, 1449, 1921, 1748], 30: [2024, 3918, 3221, 190, 3158], 31: [371, 159, 62, 383, 191], 32: [296, 47, 589, 7

### Step 4: Rating Prediction

- For each target item and user pair where the user hasn't rated the target item:
Identify the neighborhood of similar items to the target item.
- Predict the rating for the target item using a weighted average of the ratings of the items in its neighborhood, where the weights are the similarities between the items and the target item.
- Adjust the prediction based on the user's average rating or other normalization techniques, if necessary.

In [37]:
# Initialize an empty DataFrame to store predicted ratings
predicted_ratings_train = pd.DataFrame(index=train_matrix.index, columns=train_matrix.columns)

# Iterate over each user-item pair in the training set
for user_id, user_ratings in train_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train:
            neighborhood = item_neighborhoods_train[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the training set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_train.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_train.fillna(predicted_ratings_train.mean().mean(), inplace=True)

# Display the head of the predicted_ratings DataFrame
predicted_ratings_train.head()


In [ ]:
# We count the NaN values in the predicted_ratings_train dataframe
predicted_ratings_train.isnull().sum().sum()

0

In [ ]:
def get_top_recommendations(user_id, predicted_ratings, df):
    """
    Get top movie recommendations for a given user using predicted ratings.

    Parameters:
        user_id (int): ID of the user for whom recommendations are to be generated.
        predicted_ratings (pd.DataFrame): DataFrame containing predicted ratings for users and movies.
        df (pd.DataFrame): DataFrame containing movie ratings.

    Returns:
        list: Top movie titles recommended for the user.
    """
    # Ensure that predicted_ratings is a DataFrame
    if not isinstance(predicted_ratings, pd.DataFrame):
        raise ValueError("predicted_ratings must be a pandas DataFrame")

    # Check if the user ID exists in the predicted ratings DataFrame's index
    if user_id not in predicted_ratings.index:
        # If the user ID doesn't exist, return an empty list
        return ["The user doesn't exist"]

    # Get the predicted ratings for the user
    user_predicted_ratings = predicted_ratings.loc[user_id]
    
    # Filter out the movies that the user has already seen
    seen_movies = set(df[df['userId'] == user_id]['movieId'])
    unseen_movies = [movie_id for movie_id in user_predicted_ratings.index if movie_id not in seen_movies]
    
    # Check if there are unseen movies with predicted ratings
    if not unseen_movies:
        # If all movies have been seen, return an empty list
        return []
    
    # Sort the unseen movies by predicted rating in descending order
    sorted_unseen_movies = user_predicted_ratings[unseen_movies].sort_values(ascending=False)
    
    # Get the top 5 movie titles
    top_movie_ids = sorted_unseen_movies.head(5).index.tolist()
    
    # Get the unique movie titles corresponding to the top movie IDs
    top_movie_titles = set(df[df['movieId'].isin(top_movie_ids)]['title'])
    
    return list(top_movie_titles)[:5]  # Return only the first 5 unique movie titles


In [ ]:
top_recommendations = []

# Iterate over each row (user) in the train_matrix
for user_id in train_matrix.index:
    # Get the actual user ID corresponding to the user index
    recommendations = get_top_recommendations(user_id, predicted_ratings_train, df)
    top_recommendations.append(recommendations)

# Output: List of top movie recommendations for each user in the train matrix
top_recommendations

# print size of top_recommendations
len(top_recommendations)


[['How the Grinch Stole Christmas! ',
  'Sunshine ',
  'Disturbia ',
  'Vie en Rose, La (Môme, La) ',
  'Toy Story '],
 ['Closer ',
  "It's a Very Merry Muppet Christmas Movie ",
  "He's Just Not That Into You ",
  'Bridges of Madison County, The ',
  'Atonement '],
 ['Flight of the Navigator ',
  'True Lies ',
  'Rocketeer, The ',
  'Mystery of the Third Planet, The (Tayna tretey planety) ',
  'Toy Story '],
 ['Birds, The ',
  'Bad Boys ',
  'Blade Runner ',
  'Exorcist, The ',
  'Time Bandits '],
 ['Harry Potter and the Deathly Hallows: Part 1 ',
  'Alice in Wonderland ',
  'The Hobbit: The Battle of the Five Armies ',
  'Harry Potter and the Half-Blood Prince ',
  'Guest from the Future (Gostya iz buduschego) '],
 ['Excalibur ',
  'Riki-Oh: The Story of Ricky (Lik Wong) ',
  'Gamer ',
  "Heaven's Prisoners ",
  'Old Boy '],
 ['Harry Potter and the Order of the Phoenix ',
  'Sound of Thunder, A ',
  'Princess Mononoke (Mononoke-hime) ',
  'Unbreakable ',
  'Aelita: The Queen of Mars 

389

### Step 5: Model Evaluation
- We evaluate the performance of the ItemKNN algorithm using appropriate evaluation metrics such as Root Mean Squared Error (RMSE) and Mean Absolute Error (MAE)
- Split the dataset into training and testing sets to assess the model's predictive accuracy on unseen data.

In [ ]:
train.shape

predicted_ratings_train.shape

(389, 2923)

(389, 2923)

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

def evaluate_model(train_matrix, predicted_ratings):
    """
    Evaluate the model's performance on the training data.

    Parameters:
        train_matrix (numpy.ndarray): Item-user matrix from the training data.
        predicted_ratings (pandas.DataFrame or numpy.ndarray): Predicted ratings DataFrame or array for the training data.

    Returns:
        float: Mean Absolute Error (MAE) on the training data.
        float: Root Mean Squared Error (RMSE) on the training data.
    """
    # Convert predicted_ratings to a numpy array if it's a DataFrame
    if isinstance(predicted_ratings, pd.DataFrame):
        predicted_ratings = predicted_ratings.to_numpy()

    # Ensure train_matrix and predicted_ratings have the same shape
    assert train_matrix.shape == predicted_ratings.shape, "Shapes of train_matrix and predicted_ratings are not consistent."

    # Initialize lists to store true and predicted ratings
    true_ratings = []
    pred_ratings = []

    # Iterate over each user and their ratings
    for user_id, user_ratings in enumerate(train_matrix):
        for movie_id, rating in enumerate(user_ratings):
            # Skip unrated movies
            if rating == 0:
                continue

            # Check if the indices are within the bounds of the predicted ratings array
            if user_id < predicted_ratings.shape[0] and movie_id < predicted_ratings.shape[1]:
                # Get the predicted rating for the corresponding movie
                pred_rating = predicted_ratings[user_id, movie_id]

                # Append the true and predicted ratings
                true_ratings.append(rating)
                pred_ratings.append(pred_rating)

    # Convert lists to numpy arrays
    true_ratings = np.array(true_ratings)
    pred_ratings = np.array(pred_ratings)

    # Calculate evaluation metrics
    mae = mean_absolute_error(true_ratings, pred_ratings)
    rmse = np.sqrt(mean_squared_error(true_ratings, pred_ratings))

    return mae, rmse

# Assuming train is the numpy array or DataFrame representing the item-user matrix
# Assuming predicted_ratings_train is the DataFrame or numpy array representing the predicted ratings for the training data

# Evaluate the model
mae, rmse = evaluate_model(train, predicted_ratings_train)

print("Mean Absolute Error (MAE) on Training Data:", mae)
print("Root Mean Squared Error (RMSE) on Training Data:", rmse)


Mean Absolute Error (MAE) on Training Data: 0.7196990469744047
Root Mean Squared Error (RMSE) on Training Data: 0.9729005196085564


### Step 6: Parameter Tuning
- Experiment with different parameters such as similarity threshold, neighborhood size, and similarity metric to optimize the performance of the ItemKNN algorithm.
- Use cross-validation or other techniques to tune these parameters and avoid overfitting.

1. **Import Necessary Libraries:**
   - We import the required libraries for performing grid search cross-validation (`GridSearchCV`), creating custom scorers (`make_scorer`), and utilizing the `NearestNeighbors` algorithm.

2. **Define Cosine Similarity Function:**
   - We define a custom function `cosine_similarity` to compute the cosine similarity between two vectors. This function calculates the dot product of the vectors and divides it by the product of their norms.

3. **Define Custom Scorer:**
   - We create a custom scorer `cosine_similarity_scorer` using `make_scorer`, which enables us to use cosine similarity as the scoring metric during grid search cross-validation.

4. **Define Parameter Grid:**
   - We specify a parameter grid `param_grid` containing the hyperparameters to be tuned. In this case, we're tuning the number of neighbors (`n_neighbors`) and the distance metric (`metric`) for the `NearestNeighbors` algorithm.

5. **Initialize NearestNeighbors Model:**
   - We initialize the `NearestNeighbors` model without specifying any hyperparameters.

6. **Create GridSearchCV Object:**
   - We create a `GridSearchCV` object named `grid_search` with the specified parameter grid, cross-validation strategy (5-fold cross-validation), and custom scoring metric (`cosine_similarity_scorer`).

7. **Fit the Data:**
   - We fit the `item_user_matrix` data to the `grid_search` object to perform hyperparameter tuning. `item_user_matrix` typically contains the item-item similarity matrix computed using collaborative filtering techniques.

8. **Get Best Hyperparameters:**
   - After fitting the data, we retrieve the best hyperparameters selected by the grid search using the `best_params_` attribute of the `grid_search` object.

9. **Print Best Parameters:**
   - Finally, we print the best hyperparameters obtained from the grid search.



`scikit-learn (sklearn)`:

- Scikit-learn is a popular machine learning library in Python that provides simple and efficient tools for data analysis and modeling.
- It includes various modules for tasks such as classification, regression, clustering, dimensionality reduction, and model selection.
- The GridSearchCV class from scikit-learn is used for hyperparameter tuning through grid search along with cross-validation.
- The make_scorer function allows us to create a custom scoring function for use with GridSearchCV.
- The NearestNeighbors class provides functionality for unsupervised nearest neighbors learning, which can be used for tasks such as finding k-nearest neighbors for a given data point.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import make_scorer
import numpy as np

# Define a custom scorer based on cosine similarity defined above
cosine_similarity_scorer = make_scorer(cosine_similarity)

# Define the parameter grid
param_grid = {
    'n_neighbors': [5, 10, 15, 30, 40],
    'metric': ['cosine', 'euclidean']
}

# Initialize NearestNeighbors model
knn_model = NearestNeighbors()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring=cosine_similarity_scorer)

# Fit the data to perform hyperparameter tuning
grid_search.fit(train_matrix)  # train_matrix contains item-user matrix for the train set

# Get the best hyperparameters
best_params = grid_search.best_params_

print("Best parameters:", best_params)


c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_

GridSearchCV(cv=5, estimator=NearestNeighbors(),
             param_grid={'metric': ['cosine', 'euclidean'],
                         'n_neighbors': [5, 10, 15, 30, 40]},
             scoring=make_scorer(cosine_similarity, response_method='predict'))

Best parameters: {'metric': 'cosine', 'n_neighbors': 5}


This code sets up a GridSearchCV object to perform hyperparameter tuning using the train set (train_matrix). It explores different combinations of hyperparameters specified in the param_grid, evaluates them using 5-fold cross-validation (cv=5), and uses the cosine_similarity scorer to optimize the model's performance based on cosine similarity. Finally, it prints the best hyperparameters found during the search.

We have to perform the calculation of the predicted ratings for the train-val set to generate predicted ratings in order to evaluate the performance of our trained model on a validation dataset. 

If we computed the similarity matrix again specifically for the train_val set, it would essentially mean that we are using a different set of similarity measures for predicting ratings compared to what we used during training. This approach could lead to inconsistencies and potentially degrade the performance of our model. Here's what could happen:

1. **Inconsistency**: The similarity measures computed for the train_val set might differ from those computed for the training set due to variations in the data. As a result, the predicted ratings based on these new similarity measures may not align well with the predictions made during training, leading to inconsistency in the model's behavior.

2. **Overfitting**: Computing a new similarity matrix specifically for the train_val set might lead to overfitting on the validation data. The model may capture noise or idiosyncrasies present in the train_val set, which may not generalize well to unseen data.

3. **Increased Complexity**: Computing the similarity matrix again for the train_val set adds computational complexity and redundancy, especially if the similarity computation process is resource-intensive. This can result in longer training times and increased resource utilization.

Overall, it's generally recommended to use the same similarity measures or neighborhood definitions for both training and validation sets to ensure consistency and generalizability of the model.

In [ ]:
# Initialize an empty DataFrame to store predicted ratings for train_val
predicted_ratings_val = pd.DataFrame(index=val_matrix.index, columns=val_matrix.columns)

# Iterate over each user and their ratings in the train_val set
for user_id, user_ratings in val_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train:
            neighborhood = item_neighborhoods_train[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the train set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_val.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_val.fillna(predicted_ratings_val.mean().mean(), inplace=True)


In [ ]:
predicted_ratings_val.head()

predicted_ratings_val.shape

movieId,1,4,9,11,13,15,18,20,24,30,...,185473,186587,187031,187541,188675,188797,191005,193573,193583,193587
userId,,,,,,,,,,,,,,,,,,,,,
270,5.0,3.625,3.625,3.625,3.625,4.0,3.625,3.625,3.625,3.625,...,3.625,3.625,3.625,3.625,3.625,3.625,3.625,3.625,3.625,3.625
124,4.15625,4.15625,4.15625,4.15625,4.15625,4.15625,4.15625,4.15625,4.15625,4.15625,...,4.15625,4.15625,4.15625,4.15625,4.15625,4.15625,4.15625,4.15625,4.15625,4.15625
368,2.776224,2.776224,2.776224,2.776224,2.776224,3.0,2.776224,2.776224,2.776224,2.776224,...,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224,1.0,2.776224,2.776224,2.776224
213,4.0,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,...,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667
63,3.125,3.570423,3.570423,4.0,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,...,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423


(98, 2923)

Now we evaluate the model with the Validation item-user matrix.

In [ ]:
# Now, we can evaluate the model using the evaluate_model function
mae, rmse = evaluate_model(val_matrix.to_numpy(), predicted_ratings_val)

print("Mean Absolute Error (MAE) on Validation Data:", mae)
print("Root Mean Squared Error (RMSE) on Validation Data:", rmse)


Mean Absolute Error (MAE) on Validation Data: 0.7640488926024421
Root Mean Squared Error (RMSE) on Validation Data: 1.026630924440244


### Retraining of the model with the Train-Val set

In [ ]:
# After the model evaluation on the validation set, we recalculate the similarity matrix
# of the model on the train-val data
item_similarity_matrix_train_val = calculate_item_similarity_matrix(train_val)

# We combine the similarity matrices (the retrained one and the genres)
item_similarity_matrix_train_val = combine_similarity_matrices(item_similarity_matrix_train_val, item_similarity_genres_matrix)

# We calculate the item neighborhoods for the train_val data 
item_neighborhoods_train_val = neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix_train_val)

### Step 7: Deployment

Once we have come up with the best parameters possible and trained the model with the whole train_validation set, we will test it on the test set.

In [ ]:
test_matrix

movieId,1,4,9,11,13,15,18,20,24,30,...,185473,186587,187031,187541,188675,188797,191005,193573,193583,193587
userId,,,,,,,,,,,,,,,,,,,,,
293,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
367,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
469,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
119,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Check if there are NaN values in the test matrix
test_matrix.isnull().sum().sum()

0

In [ ]:
# Now, we can evaluate the model's predicted ratings on the test set

# Initialize an empty DataFrame to store predicted ratings
predicted_ratings_test = pd.DataFrame(index=test_matrix.index, columns=test_matrix.columns)

# Iterate over each user-item pair in the training set
for user_id, user_ratings in test_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
    
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train_val:
            neighborhood = item_neighborhoods_train_val[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the training set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_test.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_test.fillna(predicted_ratings_test.mean().mean(), inplace=True)

# Now, we can evaluate the model's predicted ratings on the test set
mae, rmse = evaluate_model(test_matrix.to_numpy(), predicted_ratings_test.to_numpy())

print("Mean Absolute Error (MAE) on Test Data using the Ratings given to the Test Set:", mae)
print("Root Mean Squared Error (RMSE) on Test Data using the Ratings given to the Test Set:", rmse)


Mean Absolute Error (MAE) on Test Data using the Ratings given to the Test Set: 0.7192122089072419
Root Mean Squared Error (RMSE) on Test Data using the Ratings given to the Test Set: 0.961897350285074


In [ ]:
top_recommendations_test = []

# Iterate over each row (user) in the test_matrix
for user_id in test_matrix.index:
    # Get the actual user ID corresponding to the user index
    recommendations = get_top_recommendations(user_id, predicted_ratings_test, df)
    top_recommendations_test.append(recommendations)

# Output: List of top movie recommendations for each user in the test matrix
top_recommendations_test

# print size of top_recommendations
print("Size of top_recommendations_test:", len(top_recommendations_test))



[['Shining, The ',
  'Pulp Fiction ',
  'Usual Suspects, The ',
  'Interview with the Vampire: The Vampire Chronicles ',
  'FearDotCom (a.k.a. Fear.com) (a.k.a. Fear Dot Com) '],
 ['Wallace & Gromit: The Wrong Trousers ',
  'Interview with the Vampire: The Vampire Chronicles ',
  'Spirited Away (Sen to Chihiro no kamikakushi) ',
  'Tigger Movie, The ',
  'Pirates of the Caribbean: The Curse of the Black Pearl '],
 ['Flags of Our Fathers ',
  'Dick Tracy ',
  'Jagged Edge ',
  'Missing in Action ',
  'Dumbo '],
 ['Trainspotting ',
  "Schindler's List ",
  'Quick and the Dead, The ',
  'Jackie Brown ',
  'Big Lebowski, The '],
 ['Game 6 ',
  'Little Man ',
  'Dirty Harry ',
  'A Pigeon Sat on a Branch Reflecting on Existence ',
  'Coffee and Cigarettes '],
 ['Twister ',
  'Pulp Fiction ',
  'Twelfth Night ',
  'Interview with the Vampire: The Vampire Chronicles ',
  'FearDotCom (a.k.a. Fear.com) (a.k.a. Fear Dot Com) '],
 ['Dune ', 'Trainspotting ', 'Ghost World ', 'Sunshine ', 'Time Ban

Size of top_recommendations_test: 122


# Item-based Classification (ItemKNN)

### Step 1: Data preparation
- We have to create a dataset with a user-item interactions. Each interaction should include the user ID, item ID, and the corresponding label or class for classification.
- Additionally we will create a item-genre matrix to compute the similarity between objects using the genres.
- Preprocess the data as needed, including handling missing values, encoding categorical variables, and splitting into training and testing sets.

For this step we are going to resuse the `item_user_matrix` that we created in the first model built. 

### Step 2: Compute Item Similarity


In [ ]:
item_similarity_matrix_train.shape

(2923, 2923)

In [ ]:
import numpy as np

def item_neighborhood_selection(similarity_matrix, k=None, threshold=None, item_ids=None):
    """
    Select a subset of similar items for each item based on similarity matrix.

    Parameters:
        similarity_matrix (numpy.ndarray): Item-item similarity matrix.
        k (int): Number of similar items to select (optional).
        threshold (float): Similarity threshold for selecting similar items (optional).
        item_ids (list): List of item IDs corresponding to rows/columns of the similarity matrix.

    Returns:
        dict: Dictionary containing similar items for each item.
    """
    num_items = similarity_matrix.shape[0]
    item_neighborhood = {}

    for i in range(num_items):
        if k is not None:
            # Select top-k similar items (excluding the item itself)
            similar_items_indices = np.argsort(similarity_matrix[i])[::-1][:k]
        elif threshold is not None:
            # Select items with similarity above threshold (excluding the item itself)
            similar_items_indices = np.where(similarity_matrix[i] > threshold)[0]

        # Remove the item itself from the neighborhood
        similar_items_indices = similar_items_indices[similar_items_indices != i]

        # Get the item ID corresponding to the current index
        current_item_id = item_ids[i] if item_ids is not None else i

        # Get the item IDs for the neighborhood
        neighborhood_item_ids = [item_ids[index] for index in similar_items_indices]

        # Store the similar items with the item's ID as the key
        item_neighborhood[current_item_id] = neighborhood_item_ids

    return item_neighborhood

# We have an item-item similarity matrix 'item_similarity_matrix_train'
# and we want to select top-5 similar items for each item
k = 5
item_neighborhoods_classification_train = item_neighborhood_selection(item_similarity_matrix_train, k=k, item_ids=index_to_movie_id)

# Check if there are any items with empty neighborhoods
empty_neighborhoods = []

for item_id, neighborhood in item_neighborhoods_classification_train.items():
    if not neighborhood:
        empty_neighborhoods.append(item_id)

if empty_neighborhoods:
    print("Items with empty neighborhoods:", empty_neighborhoods)
else:
    print("No items with empty neighborhoods found.")


No items with empty neighborhoods found.


In [ ]:
len(item_neighborhoods_classification_train)

2923

### Step 3: Ratings classification selection

In [ ]:
import numpy as np
import pandas as pd

def recommend_movies(train_matrix, item_neighborhoods, movie_id_to_index, num_movies=5):
    """
    Recommend top movies for each user based on item neighborhood.

    Parameters:
        train_matrix (np.ndarray or pd.DataFrame): Matrix containing user-item ratings in the train set.
        item_neighborhoods (dict): Dictionary containing similar items for each item.
        movie_id_to_index (dict): Dictionary mapping movie IDs to their corresponding indices in the train_matrix.
        num_movies (int): Number of movies to recommend for each user.

    Returns:
        pd.DataFrame: DataFrame containing predicted ratings for recommended movies for each user.
    """
    # Convert train_matrix to DataFrame if it's a numpy array
    if isinstance(train_matrix, np.ndarray):
        train_matrix = pd.DataFrame(train_matrix, index=np.arange(train_matrix.shape[0]), columns=np.arange(train_matrix.shape[1]))

    # Initialize an empty DataFrame to store predicted ratings
    predicted_ratings = pd.DataFrame(index=train_matrix.index, columns=train_matrix.columns)

    # Iterate over each user-item pair in the training set
    for user_id, user_ratings in train_matrix.iterrows():
        for movie_id, rating in user_ratings.items():
            
            # Skip if the rating is non-zero (indicating a rating given by the user)
            if rating != 0:
                continue
            
            # Check if the movie has a neighborhood defined
            if movie_id in item_neighborhoods:
                neighborhood = item_neighborhoods[movie_id]
                
                # Initialize a flag to track if the rating has been predicted
                rating_predicted = False
                
                # Iterate over the items in the neighborhood
                for neighbor_movie_id in neighborhood:
                    # Check if the neighbor movie has been rated by the user
                    if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0:
                        # Use the rating of the closest item in the neighborhood
                        predicted_rating = user_ratings.loc[neighbor_movie_id]
                        rating_predicted = True
                        break
                
                # If none of the items in the neighborhood have been rated, use the user's average rating
                if not rating_predicted:
                    if user_ratings[user_ratings != 0].empty:
                        # If the user hasn't rated any movies, use the global mean rating
                        predicted_rating = train_matrix[train_matrix != 0].mean().mean()
                    else:
                        # Use the average of the ratings given by the user
                        predicted_rating = user_ratings[user_ratings != 0].mean()
                
                # Assign the predicted rating to the corresponding cell in the DataFrame
                predicted_ratings.at[user_id, movie_id] = predicted_rating

    # Fill NaN values with mean ratings across all users
    predicted_ratings.fillna(predicted_ratings.mean().mean(), inplace=True)

    # Set the index of the DataFrame as the user ID of the train set
    predicted_ratings.index = train_matrix.index

    # Display the head of the predicted_ratings DataFrame
    print(predicted_ratings.head())

    return predicted_ratings


# Recommend movies for each user in the train set
predicted_ratings_classification_train = recommend_movies(train_matrix, item_neighborhoods_classification_train, movie_id_to_index)

# Print the shape of the predicted ratings matrix
print("Shape of predicted ratings matrix:", predicted_ratings_classification_train.shape)


In [ ]:
type(train_matrix)

pandas.core.frame.DataFrame

In [ ]:
predicted_ratings_classification_train

movieId,1,4,15,30,43,89,104,108,122,146,...,174479,174551,175475,176371,176389,177593,179813,181413,185029,186587
userId,,,,,,,,,,,,,,,,,,,,,
428,1.000000,2.384615,2.384615,2.384615,2.384615,2.384615,3.604726,2.384615,2.384615,2.384615,...,2.384615,2.384615,2.384615,2.384615,2.384615,2.384615,2.384615,1.000000,2.384615,2.384615
517,3.604726,2.411765,2.411765,2.411765,2.411765,2.411765,2.411765,2.411765,0.500000,2.411765,...,2.411765,2.411765,2.411765,2.411765,4.000000,2.411765,2.411765,4.000000,2.411765,2.411765
197,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
160,3.604726,3.047619,3.047619,3.047619,3.047619,2.000000,3.047619,3.047619,3.047619,3.047619,...,3.047619,4.000000,3.047619,3.047619,4.000000,3.047619,3.047619,4.000000,3.047619,3.047619
67,3.500000,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,...,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,3.500000,4.166667,4.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,5.000000,4.305556,4.305556,4.305556,4.305556,4.305556,5.000000,4.305556,4.305556,4.305556,...,4.305556,4.305556,4.305556,4.305556,4.305556,4.305556,4.305556,4.305556,4.305556,4.305556
267,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
326,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,3.500000,4.000000,4.000000,4.000000,4.000000


### Step 4: Recommendations Generation

In [ ]:
import pandas as pd

# Convert predicted_ratings_classification_train to a pandas DataFrame if it's a numpy array
if isinstance(predicted_ratings_classification_train, np.ndarray):
    # Use the length of train_matrix as the number of rows and items as columns
    num_users, num_items = train_matrix.shape
    predicted_ratings_classification_train = pd.DataFrame(predicted_ratings_classification_train, index=range(num_users), columns=range(num_items))

# Initialize a dictionary to store top recommendations for each user
top_recommendations_per_user = {}

# Iterate over each user in the train matrix
for user_id in train_matrix.index:
    # Get the top recommendations for the current user using predicted_ratings_classification_train
    recommendations = get_top_recommendations(user_id, predicted_ratings_classification_train, df)
    
    # Store the recommendations in the dictionary
    top_recommendations_per_user[user_id] = recommendations

# Print the top 5 recommendations for each user
for user_id, recommendations in top_recommendations_per_user.items():
    print(f"User {user_id} recommendations:", recommendations)


User 428 recommendations: ['How the Grinch Stole Christmas! ', 'Darkon ', 'We Could Be King ', 'Starsky & Hutch ', "Outfoxed: Rupert Murdoch's War on Journalism "]
User 517 recommendations: ['Life Is Beautiful (La Vita è bella) ', "Can't Stop the Music ", 'Sandpiper, The ', 'Funny Face ', 'Fallen Angels (Duo luo tian shi) ']
User 197 recommendations: ['All the Boys Love Mandy Lane ', 'Tristram Shandy: A Cock and Bull Story ', "Nina's Heavenly Delights ", 'Heart of a Dog (Sobachye serdtse) ', 'Toy Story ']
User 160 recommendations: ['Fried Green Tomatoes ', 'How the Grinch Stole Christmas! ', 'Captain Newman, M.D. ', 'Chorus Line, A ', 'Jimmy Hollywood ']
User 67 recommendations: ['All the Boys Love Mandy Lane ', 'Waitress ', 'Spy Next Door, The ', "Nina's Heavenly Delights ", '42nd Street ']
User 32 recommendations: ['Godzilla ', 'Star Trek V: The Final Frontier ', 'Mexican, The ', 'Men in Black (a.k.a. MIB) ', 'Time Bandits ']
User 234 recommendations: ['What Women Want ', "It's a Won

### Step 5: Model Evaluation (Item KNN Classification)

In [ ]:
import numpy as np
import pandas as pd

# Ensure predicted_ratings_classification_train is a numpy array
if isinstance(predicted_ratings_classification_train, pd.DataFrame):
    predicted_ratings_classification_train = predicted_ratings_classification_train.to_numpy()

# Assuming train_matrix is the numpy array representing the item-user matrix

# Ensure train_matrix is a 2D array
if isinstance(train_matrix, pd.DataFrame):
    train_matrix = train_matrix.to_numpy()

if train_matrix.ndim == 1:
    train_matrix = np.expand_dims(train_matrix, axis=0)

# Evaluate the model
mae, rmse = evaluate_model(train, predicted_ratings_classification_train)

print("Mean Absolute Error (MAE) on Training Data:", mae)
print("Root Mean Squared Error (RMSE) on Training Data:", rmse)


Mean Absolute Error (MAE) on Training Data: 0.822796188644207
Root Mean Squared Error (RMSE) on Training Data: 1.0535259015044494


### Step 6: Parameter Tuning

Experiment with different parameters such as similarity threshold, neighborhood size, and similarity metric to optimize the performance of the ItemKNN algorithm.
Use techniques like cross-validation to tune these parameters and avoid overfitting.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import make_scorer
import numpy as np

# Define a custom scorer based on cosine similarity defined above
cosine_similarity_scorer = make_scorer(cosine_similarity)

# Define the parameter grid
param_grid = {
    'n_neighbors': [5, 10, 15, 30, 40],
    'metric': ['cosine', 'euclidean']
}

# Initialize NearestNeighbors model
knn_model = NearestNeighbors()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring=cosine_similarity_scorer)

# Fit the data to perform hyperparameter tuning
grid_search.fit(train_matrix)  # val_matrix contains item-user matrix

# Get the best hyperparameters
best_params = grid_search.best_params_

print("Best parameters:", best_params)


c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_

GridSearchCV(cv=5, estimator=NearestNeighbors(),
             param_grid={'metric': ['cosine', 'euclidean'],
                         'n_neighbors': [5, 10, 15, 30, 40]},
             scoring=make_scorer(cosine_similarity, response_method='predict'))

Best parameters: {'metric': 'cosine', 'n_neighbors': 5}


We are now going to obtain again the ratings for the full val set to evaluate the performance. 

In [ ]:
# Function to recalculate predictions for a given matrix using the trained model
def recalculate_predictions(matrix, item_neighborhoods):
    """
    Recalculate predictions for a given matrix using the trained model.

    Parameters:
        matrix (pd.DataFrame): Matrix containing user-item ratings.

    Returns:
        pd.DataFrame: DataFrame containing recalculated predicted ratings for each user.
    """
    # Initialize an empty DataFrame to store recalculated predicted ratings
    recalculated_predictions = pd.DataFrame(index=matrix.index, columns=matrix.columns)

    # Iterate over each user-item pair in the matrix
    for user_id, user_ratings in matrix.iterrows():
        for movie_id, rating in user_ratings.items():
            # Skip if the rating is non-zero (indicating a rating given by the user)
            if rating != 0:
                continue

            # Check if the movie has a neighborhood defined
            if movie_id in item_neighborhoods:
                neighborhood = item_neighborhoods[movie_id]

                # Initialize a flag to track if the rating has been predicted
                rating_predicted = False

                # Iterate over the items in the neighborhood
                for neighbor_movie_id in neighborhood:
                    # Check if the neighbor movie has been rated by the user
                    if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0:
                        # Use the rating of the closest item in the neighborhood
                        predicted_rating = user_ratings.loc[neighbor_movie_id]
                        rating_predicted = True
                        break

                # If none of the items in the neighborhood have been rated, use the user's average rating
                if not rating_predicted:
                    if user_ratings[user_ratings != 0].empty:
                        # If the user hasn't rated any movies, use the global mean rating
                        predicted_rating = matrix[matrix != 0].mean().mean()
                    else:
                        # Use the average of the ratings given by the user
                        predicted_rating = user_ratings[user_ratings != 0].mean()

                # Assign the predicted rating to the corresponding cell in the DataFrame
                recalculated_predictions.at[user_id, movie_id] = predicted_rating

    # Fill NaN values with mean ratings across all users
    recalculated_predictions.fillna(recalculated_predictions.mean().mean(), inplace=True)

    return recalculated_predictions


# Recalculate predictions for the validation set (train_val_matrix)
recalculated_val_predictions = recalculate_predictions(val_matrix, item_neighborhoods_classification_train)


C:\Users\Jaume\AppData\Local\Temp\ipykernel_2140\4246365781.py:54: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  recalculated_predictions.fillna(recalculated_predictions.mean().mean(), inplace=True)


In [ ]:
recalculated_val_predictions

movieId,1,4,15,30,43,89,104,108,122,146,...,174479,174551,175475,176371,176389,177593,179813,181413,185029,186587
userId,,,,,,,,,,,,,,,,,,,,,
536,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
577,3.444444,3.444444,3.444444,3.444444,3.444444,3.444444,4.000000,3.444444,3.444444,3.444444,...,3.444444,3.444444,3.444444,3.444444,3.444444,3.444444,3.444444,3.444444,3.444444,3.444444
113,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,...,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000
448,3.652444,2.829670,3.000000,2.829670,2.829670,4.000000,3.652444,2.829670,3.652444,2.829670,...,2.829670,4.000000,2.829670,2.829670,5.000000,2.829670,2.829670,2.829670,2.829670,2.000000
523,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,...,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667,4.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,...,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000
605,3.652444,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,4.000000,3.000000,3.000000,3.500000,3.000000,3.000000
79,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,...,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000


In [ ]:
# Evaluate the model
mae, rmse = evaluate_model(val, recalculated_val_predictions)

print("Mean Absolute Error (MAE) on Train-Validation Data:", mae)
print("Root Mean Squared Error (RMSE) on Train-Validation Data:", rmse)

Mean Absolute Error (MAE) on Train-Validation Data: 0.8862384903582982
Root Mean Squared Error (RMSE) on Train-Validation Data: 1.1062703799857017


### Step 7: Deployment

Once we have all the metrics, we have to recalculate the ratings for the test set, we are going to repeat the same process for the test set.

In [ ]:
# We have to reobtain the neighborhood for the train_val data
# We will use the already calculated item-item similarity matrix for train_val data
item_neighborhoods_classification_train_val = item_neighborhood_selection(item_similarity_matrix_train_val, k=5, item_ids=index_to_movie_id)

# Recalculate predictions for the validation set (test matrix)
recalculated_test_predictions = recalculate_predictions(test_matrix, item_neighborhoods_classification_train_val)

C:\Users\Jaume\AppData\Local\Temp\ipykernel_2140\4246365781.py:54: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  recalculated_predictions.fillna(recalculated_predictions.mean().mean(), inplace=True)


We also have to evaluate the metrics for the test set.

In [ ]:
# Evaluate the model
mae, rmse = evaluate_model(test, recalculated_test_predictions)

print("Mean Absolute Error (MAE) on Test Data:", mae)
print("Root Mean Squared Error (RMSE) on Test Data:", rmse)

Mean Absolute Error (MAE) on Test Data: 0.809245626540846
Root Mean Squared Error (RMSE) on Test Data: 1.0033399816659307
